In [9]:
from standardInclude import *

In [10]:
#f = r.TFile("./data/stripped_gm2skim_ana_16356.root")
#f.cd("trackerNTup")
#f.ls()

#t = f.Get("allmuons/tree")

t = r.TChain("allmuons/tree")
t.Add("./data/stripped_gm2skim_ana_16356.root")

#t.Print()

1

In [11]:
r.globalSeed = 1234

In [12]:
print(r.globalSeed)

1234


In [ ]:
r.gInterpreter.ProcessLine('.L testfunc.C')

0

In [ ]:
r.initializeRNG(1234)

<ROOT.TRandom3 object ("Random3") at 0x7fffdc7c0360>

In [ ]:
r.testfunc(8)

19921.445622315165

---

### Test of histogram time randomization when filling from the tree, based on code from James M.

---

In [ ]:
def randomizeTimes( x, p ):
    inputtime = x[0]
    time = inputtime
    
    w0 = 2.6094;
    A = 2.80;
    tauA = 56.6;
    B = 6.18;
    tauB = 6.32;
    
    wCBO =  (w0 - (A/(tauA)) * r.TMath.Exp(-time/(1000.0*tauA)) - (B/(tauB)) * r.TMath.Exp(-time/(tauB*1000.0)) ); #MHz
    wa =  r.TMath.TwoPi() * 0.2291; #MHZ
    wc =  r.TMath.TwoPi() / 0.14919; #MHz

    factor = 1.0;
    wY = factor * (wCBO) * r.TMath.Sqrt( (2 * wc / (factor * wCBO)) -1.0 ) ;
    wVW = wc - (2.0 * wY);

    TCBO = r.TMath.TwoPi() / wCBO;
    Ta = r.TMath.TwoPi() / wa;
    TVW = r.TMath.TwoPi() / wVW;
    TY = r.TMath.TwoPi() / wY;
    T_diff= r.TMath.TwoPi()/(wCBO-wa);
    
    #time += (rng->Uniform()-0.5)*(Ta*1000.0);// w_a 4365
    timestorandomize = [
                        Ta, 
                        TCBO, 
                        TVW, 
                        TY, 
                        T_diff 
                       ]
    for periodi in timestorandomize:
        time += (np.random.random() - 0.5)*(periodi)
    #print(inputtime, time, TCBO, Ta)
    
    return time

In [ ]:
h = r.TH1D("h","Test wiggle no random",4700,0,700)
h2 = h.Clone("h2")
h2.SetTitle("Test wiggle random")
h3 = h2.Clone("h3")
h3.SetLineColor(3)

In [ ]:
h2.SetLineColor(2)
h2.SetMarkerColor(2)

In [ ]:
caloEnergyCut = "cluEne > 1700 && cluEne < 3200 "
qualityCut = "" #"trkPassCaloVertexQuality && trkPassTrackQuality"

allCuts = caloEnergyCut#+" && "+qualityCut

nclusters = t.Draw("cluTime/1000*1.25>>h",allCuts,"goff")

In [ ]:
for i in range(nclusters):
    h2.Fill(randomizeTimes([t.GetV1()[i]], [1]))

In [ ]:
nclusters = t.Draw("testfunc(cluTime)/1000*1.25>>h3",allCuts,"goff")

In [ ]:
c = r.TCanvas()
h.Draw()
h2.Draw("SAME")
h3.Draw("SAME")
c.SetLogy()
h.GetXaxis().SetRangeUser(30,50)
c.Draw()